# Logistic Regression Consulting Project

## Binary Customer Churn

A marketing agency has many customers that use their service to produce ads for the client/customer websites. They've noticed that they have quite a bit of churn in clients. They basically randomly assign account managers right now, but want you to create a machine learning model that will help predict which customers will churn (stop buying their service) so that they can correctly assign the customers most at risk to churn an account manager. Luckily they have some historical data, can you help them out? Create a classification algorithm that will help classify whether or not a customer churned. Then the company can test this against incoming data for future customers to predict which customers will churn and assign them an account manager.

The data is saved as customer_churn.csv. Here are the fields and their definitions:

    Name : Name of the latest contact at Company
    Age: Customer Age
    Total_Purchase: Total Ads Purchased
    Account_Manager: Binary 0=No manager, 1= Account manager assigned
    Years: Totaly Years as a customer
    Num_sites: Number of websites that use the service.
    Onboard_date: Date that the name of the latest contact was onboarded
    Location: Client HQ Address
    Company: Name of Client Company
    
Once you've created the model and evaluated it, test out the model on some new data (you can think of this almost like a hold-out set) that your client has provided, saved under new_customers.csv. The client wants to know which customers are most likely to churn given this data (they don't have the label yet).

In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('logr_cons').getOrCreate()

In [0]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import VectorAssembler, VectorIndexer, OneHotEncoder, StringIndexer
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.sql.functions import year, month, dayofmonth

In [0]:
df = sqlContext.sql("SELECT * FROM customer_churn_1_csv")
df.describe().show()

+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+--------------------+--------------------+-------------------+
summary| Names| Age| Total_Purchase| Account_Manager| Years| Num_Sites| Location| Company| Churn|
+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+--------------------+--------------------+-------------------+
 count| 900| 900| 900| 900| 900| 900| 900| 900| 900|
 mean| null|41.81666666666667|10062.82403333334|0.4811111111111111| 5.27315555555555| 8.587777777777777| null| null|0.16666666666666666|
 stddev| null|6.127560416916251|2408.644531858096|0.4999208935073339|1.274449013194616|1.7648355920350969| null| null| 0.3728852122772358|
 min| Aaron King| 22.0| 100.0| 0| 1.0| 3.0|00103 Jeffrey Cre...| Abbott-Thompson| 0|
 max|Zachary Walsh| 65.0| 18026.01| 1| 9.15| 14.0|Unit 9800 Box 287...|Zuniga, Clark and...| 1|
+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+--------------------+--------------------+-------------------+

In [0]:
df.columns

Out[20]: ['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

In [0]:
df.groupBy('Location').count().show()


+--------------------+-----+
 Location|count|
+--------------------+-----+
062 Trevor Falls ...| 1|
066 Jenkins Walks...| 1|
45946 Day Springs...| 1|
143 Andrea Flat L...| 1|
Unit 2093 Box 153...| 1|
399 Herbert Key P...| 1|
104 Ruben Rapid A...| 1|
930 Carrie Harbor...| 1|
8202 Jade Unions ...| 1|
USCGC Bailey FPO ...| 1|
893 Carla Trace S...| 1|
446 Rodney Ridge ...| 1|
30668 Isabella Fr...| 1|
911 Kent Point An...| 1|
078 Nunez Haven S...| 1|
PSC 5667, Box 831...| 1|
4972 Michael Vill...| 1|
567 Ian Loop Lamb...| 1|
482 Wells Mountai...| 1|
7259 Brown Street...| 1|
+--------------------+-----+
only showing top 20 rows

In [0]:
myCols = [
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Churn']
cdf = df.select(myCols)
cdf.show()

+----+--------------+---------------+-----+---------+-----+
 Age|Total_Purchase|Account_Manager|Years|Num_Sites|Churn|
+----+--------------+---------------+-----+---------+-----+
42.0| 11066.8| 0| 7.22| 8.0| 1|
41.0| 11916.22| 0| 6.5| 11.0| 1|
38.0| 12884.75| 0| 6.67| 12.0| 1|
42.0| 8010.76| 0| 6.71| 10.0| 1|
37.0| 9191.58| 0| 5.56| 9.0| 1|
48.0| 10356.02| 0| 5.12| 8.0| 1|
44.0| 11331.58| 1| 5.23| 11.0| 1|
32.0| 9885.12| 1| 6.92| 9.0| 1|
43.0| 14062.6| 1| 5.46| 11.0| 1|
40.0| 8066.94| 1| 7.11| 11.0| 1|
30.0| 11575.37| 1| 5.22| 8.0| 1|
45.0| 8771.02| 1| 6.64| 11.0| 1|
45.0| 8988.67| 1| 4.84| 11.0| 1|
40.0| 8283.32| 1| 5.1| 13.0| 1|
41.0| 6569.87| 1| 4.3| 11.0| 1|
38.0| 10494.82| 1| 6.81| 12.0| 1|
45.0| 8213.41| 1| 7.35| 11.0| 1|
43.0| 11226.88| 0| 8.08| 12.0| 1|
53.0| 5515.09| 0| 6.85| 8.0| 1|
46.0| 8046.4| 1| 5.69| 8.0| 1|
+----+--------------+---------------+-----+---------+-----+
only showing top 20 rows

In [0]:
fdf = cdf.na.drop()
fdf.columns

Out[41]: ['Age', 'Total_Purchase', 'Account_Manager', 'Years', 'Num_Sites', 'Churn']

In [0]:
ass = VectorAssembler(inputCols=['Age', 'Total_Purchase', 'Years', 'Num_Sites'],outputCol='features')
opx = ass.transform(fdf)
op = opx.select('features','Churn')

In [0]:
train, test = op.randomSplit([.7,.3])

In [0]:
lrModel = LogisticRegression(labelCol='Churn')


In [0]:
fitted = lrModel.fit(train)

In [0]:
fitted.summary.predictions.describe().show()

+-------+-------------------+-------------------+
summary| Churn| prediction|
+-------+-------------------+-------------------+
 count| 631| 631|
 mean|0.17274167987321712|0.11885895404120443|
 stddev| 0.3783236979973364| 0.3238792113836936|
 min| 0.0| 0.0|
 max| 1.0| 1.0|
+-------+-------------------+-------------------+

In [0]:
res = fitted.evaluate(test)
res.predictions.show()

+--------------------+-----+--------------------+--------------------+----------+
 features|Churn| rawPrediction| probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
[28.0,8670.98,3.9...| 0|[7.24356559342806...|[0.99928575184271...| 0.0|
[28.0,11128.95,5....| 0|[4.18078226567064...|[0.98494361534862...| 0.0|
[28.0,11204.23,3....| 0|[1.51583538212969...|[0.81992440058446...| 0.0|
[29.0,9617.59,5.4...| 0|[3.97515669048192...|[0.98156969484217...| 0.0|
[29.0,11274.46,4....| 0|[4.51417574328056...|[0.98916603125679...| 0.0|
[29.0,13255.05,4....| 0|[4.17580229247822...|[0.98486958526018...| 0.0|
[30.0,6744.87,5.1...| 0|[3.06952456163968...|[0.95561801237868...| 0.0|
[30.0,8403.78,4.1...| 0|[5.90472911461795...|[0.99728089344391...| 0.0|
[30.0,8677.28,7.3...| 0|[4.08475471389834...|[0.98345120468149...| 0.0|
[30.0,10183.98,5....| 0|[2.93645055723324...|[0.94961918302632...| 0.0|
[30.0,11575.37,5....| 1|[3.99944938590967...|[0.98200406212141...| 0.0|
[31.0,5387.75,6.8...| 0|[2.10686260319799...|[0.89156840035340...| 0.0|
[31.0,10182.6,3.7...| 0|[4.82479874387081...|[0.99203576877152...| 0.0|
[31.0,11743.24,5....| 0|[6.21856854677297...|[0.99801186578400...| 0.0|
[31.0,12264.68,5....| 0|[3.54366299187165...|[0.97190490588750...| 0.0|
[32.0,6367.22,2.8...| 0|[3.12330000630463...|[0.95784368116378...| 0.0|
[32.0,7896.65,7.1...| 0|[2.95355210009248...|[0.95043110235090...| 0.0|
[32.0,8575.71,4.2...| 0|[3.39223371753839...|[0.96746093620326...| 0.0|
[32.0,11540.86,3....| 0|[6.31914196884652...|[0.99820175094852...| 0.0|
[32.0,11715.72,4....| 0|[3.00330739406607...|[0.95272332038371...| 0.0|
+--------------------+-----+--------------------+--------------------+----------+
only showing top 20 rows

In [0]:
eval = BinaryClassificationEvaluator(rawPredictionCol='prediction', labelCol='Churn')

In [0]:
res.select('Churn','Prediction').show()

+-----+----------+
Churn|Prediction|
+-----+----------+
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 1| 0.0|
 0| 1.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
+-----+----------+
only showing top 20 rows

In [0]:
auc = eval.evaluate(res.predictions)
auc

Out[70]: 0.7973363286264441

In [0]:
### new data

In [0]:
flrm = lrModel.fit(op)

In [0]:
ncust = sqlContext.sql("SELECT * FROM new_customers_csv")
ncust.describe().show()

+-------+-------------+------------------+-----------------+------------------+-----------------+------------------+-------------------+--------------------+----------------+
summary| Names| Age| Total_Purchase| Account_Manager| Years| Num_Sites| Onboard_date| Location| Company|
+-------+-------------+------------------+-----------------+------------------+-----------------+------------------+-------------------+--------------------+----------------+
 count| 6| 6| 6| 6| 6| 6| 6| 6| 6|
 mean| null|35.166666666666664|7607.156666666667|0.8333333333333334|6.808333333333334|12.333333333333334| null| null| null|
 stddev| null| 15.71517313511584|4346.008232825459| 0.408248290463863|3.708737880555414|3.3862466931200785| null| null| null|
 min|Andrew Mccall| 22.0| 100.0| 0| 1.0| 8.0|2006-12-11 07:48:13|085 Austin Views ...|Barron-Robertson|
 max| Taylor Young| 65.0| 13147.71| 1| 10.0| 15.0|2016-10-28 05:32:13|Unit 0789 Box 073...| Wood LLC|
+-------+-------------+------------------+-----------------+------------------+-----------------+------------------+-------------------+--------------------+----------------+

In [0]:
nopx = ass.transform(ncust)
nop = nopx.select('features')
nop.show()

+--------------------+
 features|
+--------------------+
[37.0,9935.53,7.7...|
[23.0,7526.94,9.2...|
[65.0,100.0,1.0,1...|
[32.0,6487.5,9.4,...|
[32.0,13147.71,10...|
[22.0,8445.26,3.4...|
+--------------------+

In [0]:
nres = fitted.transform(nop)
nres.select('prediction').show()

+----------+
prediction|
+----------+
 0.0|
 1.0|
 1.0|
 1.0|
 0.0|
 1.0|
+----------+